In [36]:
# Required Libraries
import os
import numpy as np
import pandas as pd
from obspy import read
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from scipy import signal
from datetime import timedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [37]:
# Function to convert relative time to absolute time
def convert_rel_to_abs_time(start_time, time_rel):
    """
    Convert relative time to absolute time using the trace start time.
    """
    return (start_time + timedelta(seconds=float(time_rel))).strftime('%Y-%m-%dT%H:%M:%S.%f')

# Function to apply a bandpass filter to a seismic trace
def apply_bandpass_filter(trace, lowcut=0.01, highcut=3.0, sampling_rate=6.625, order=4):
    """
    Apply a bandpass filter to seismic trace to remove noise outside the frequency range.
    """
    sos = signal.butter(order, [lowcut, highcut], btype='bandpass', fs=sampling_rate, output='sos')
    filtered_trace = signal.sosfilt(sos, trace)
    return filtered_trace

# STA/LTA Feature Extraction
def extract_sta_lta_features(trace, sampling_rate, sta_window=1.0, lta_window=5.0):
    """
    Extract STA/LTA features to detect seismic events.
    """
    sta_samples = int(sta_window * sampling_rate)
    lta_samples = int(lta_window * sampling_rate)
    cft = classic_sta_lta(trace, sta_samples, lta_samples)
    
    return cft  # STA/LTA characteristic function

# Normalization/Standardization
def normalize_trace(trace, method='standard'):
    """
    Normalize or standardize seismic data.
    
    - 'standard': standardizes the data (mean = 0, std = 1)
    - 'minmax': normalizes the data to range [0, 1]
    """
    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'minmax':
        scaler = MinMaxScaler()
    else:
        raise ValueError("Invalid normalization method. Choose 'standard' or 'minmax'.")
    
    trace = trace.reshape(-1, 1)  # Reshape trace for scaler
    trace_normalized = scaler.fit_transform(trace).flatten()  # Normalize and flatten back
    return trace_normalized

In [38]:
# Function to segment seismic events
def segment_seismic_events(trace, segment_length=3600, sampling_rate=6.625):
    """
    Segment seismic trace into smaller parts based on segment length (in seconds).
    
    Parameters:
    - trace: Seismic data (1-hour or 1-day trace).
    - segment_length: Length of each segment in seconds.
    - sampling_rate: Sampling rate of the data (samples per second).
    
    Returns:
    - List of segmented traces.
    """
    samples_per_segment = int(segment_length * sampling_rate)
    num_segments = len(trace) // samples_per_segment
    
    segmented_traces = []
    for i in range(num_segments):
        start = i * samples_per_segment
        end = (i + 1) * samples_per_segment
        segmented_traces.append(trace[start:end])
    
    return segmented_traces

In [39]:
def preprocess_seismic_data(filepath, filetype, sampling_rate=6.625, normalization_method='standard'):
    """
    Preprocess seismic data from .csv or .mseed files:
    - Apply bandpass filter
    - Extract both time-domain and frequency-domain features
    - Normalize the filtered trace
    """
    # Read the trace from file
    if filetype == 'csv':
        seismic_data = pd.read_csv(filepath)
        trace = seismic_data['velocity(c/s)'].values
        time_rel = seismic_data['rel_time(sec)'].values
    elif filetype == 'mseed':
        st = read(filepath)
        tr = st[0]
        trace = tr.data
        time_rel = tr.times()

    # Apply bandpass filter
    filtered_trace = apply_bandpass_filter(trace, sampling_rate=sampling_rate)

    # Normalize the filtered trace
    normalized_trace = normalize_trace(filtered_trace, method=normalization_method)

    # Extract time-domain features
    sta_lta_features, envelope, velocity, acceleration = extract_time_domain_features(normalized_trace, sampling_rate)

    # Extract frequency-domain features
    frequencies, times, spectrogram_data, fft_values, frequencies_psd, psd = extract_frequency_domain_features(normalized_trace, sampling_rate)

    # Convert relative times to absolute times (for labeled data)
    if filetype == 'mseed':
        starttime = tr.stats.starttime.datetime
        time_abs = [convert_rel_to_abs_time(starttime, rel_time) for rel_time in time_rel]
    else:
        time_abs = time_rel  # For CSV, time_rel is already present.
    
    return (normalized_trace, sta_lta_features, envelope, velocity, acceleration, 
            frequencies, times, spectrogram_data, fft_values, frequencies_psd, psd, time_abs)


In [40]:
from scipy.signal import hilbert

# Function to compute the envelope of a seismic signal
def compute_envelope(trace):
    """
    Compute the envelope of the signal using Hilbert transform.
    """
    analytic_signal = hilbert(trace)
    envelope = np.abs(analytic_signal)
    return envelope


In [41]:
# Function to compute the first and second derivative (velocity and acceleration)
def compute_derivatives(trace, sampling_rate):
    """
    Compute velocity (1st derivative) and acceleration (2nd derivative) of the seismic signal.
    """
    dt = 1 / sampling_rate
    velocity = np.gradient(trace, dt)  # 1st derivative
    acceleration = np.gradient(velocity, dt)  # 2nd derivative
    return velocity, acceleration


In [42]:
from scipy.signal import spectrogram

def compute_spectrogram(trace, sampling_rate, nperseg=256, noverlap=128):
    """
    Compute the spectrogram of the seismic signal to extract time-frequency features.
    """
    frequencies, times, Sxx = spectrogram(trace, fs=sampling_rate, nperseg=nperseg, noverlap=noverlap)
    return frequencies, times, Sxx


In [43]:
from scipy.fft import fft

def compute_fft(trace):
    """
    Compute the Fast Fourier Transform (FFT) of the seismic signal.
    """
    fft_values = np.abs(fft(trace))
    return fft_values


In [44]:
from scipy.signal import welch

def compute_psd(trace, sampling_rate):
    """
    Compute the Power Spectral Density (PSD) using Welch's method.
    """
    frequencies, psd = welch(trace, fs=sampling_rate, nperseg=256)
    return frequencies, psd


In [45]:
def extract_time_domain_features(trace, sampling_rate):
    """
    Extract time-domain features from the seismic trace.
    Features:
    - STA/LTA ratio
    - Envelope (amplitude)
    - Velocity and Acceleration (derivatives)
    """
    # STA/LTA feature (already done)
    sta_lta_features = extract_sta_lta_features(trace, sampling_rate)

    # Envelope (signal amplitude)
    envelope = compute_envelope(trace)

    # Velocity and Acceleration (derivatives)
    velocity, acceleration = compute_derivatives(trace, sampling_rate)

    return sta_lta_features, envelope, velocity, acceleration


In [46]:
def extract_frequency_domain_features(trace, sampling_rate):
    """
    Extract frequency-domain features from the seismic trace.
    Features:
    - Spectrogram
    - Fourier Transform (FFT)
    - Power Spectral Density (PSD)
    """
    # Spectrogram (time-frequency analysis)
    frequencies, times, spectrogram_data = compute_spectrogram(trace, sampling_rate)

    # FFT (frequency-domain analysis)
    fft_values = compute_fft(trace)

    # Power Spectral Density (PSD)
    frequencies_psd, psd = compute_psd(trace, sampling_rate)

    return frequencies, times, spectrogram_data, fft_values, frequencies_psd, psd


In [47]:
def load_lunar_data(catalog_path, data_dir):
    """
    Load lunar seismic data from the catalog and associated .mseed files.
    Only .mseed files are loaded, and the catalog is used for labeling.
    
    Parameters:
    - catalog_path: Path to the lunar catalog CSV file.
    - data_dir: Directory containing the lunar .mseed files.
    
    Returns:
    - lunar_data: List of loaded seismic traces.
    - lunar_labels: List of labels (from the catalog).
    - lunar_arrival_times: List of arrival times (absolute).
    """
    catalog = pd.read_csv(catalog_path)
    lunar_data = []
    lunar_labels = []
    lunar_arrival_times = []
    
    for _, row in catalog.iterrows():
        filename = row['filename'] + '.mseed'
        file_path = os.path.join(data_dir, filename)
        
        if os.path.exists(file_path):
            print(f"Loading MSEED file: {file_path}")
            st = read(file_path)
            trace = st[0].data
            lunar_data.append(trace)  # Extract the trace data
            lunar_labels.append(row['mq_type'])  # Label from the catalog
            lunar_arrival_times.append(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])  # Arrival time from catalog
        else:
            print(f"File {filename} not found.")
    
    return lunar_data, lunar_labels, lunar_arrival_times


In [48]:
def load_martian_data(data_dir):
    """
    Load martian seismic data from the .mseed files (unlabeled).
    
    Parameters:
    - data_dir: Directory containing the martian .mseed files.
    
    Returns:
    - martian_data: List of processed seismic traces and features.
    - martian_time_abs: List of absolute times corresponding to seismic events.
    """
    martian_data = []
    martian_time_abs = []

    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.mseed'):
                file_path = os.path.join(root, file)
                print(f"Loading MSEED file: {file_path}")

                # Adjusting to capture all returned values from preprocess_seismic_data
                (
                    filtered_trace, sta_lta_features, envelope, velocity, acceleration, 
                    frequencies, times, spectrogram_data, fft_values, frequencies_psd, psd, time_abs
                ) = preprocess_seismic_data(file_path, filetype='mseed')
                
                # Store all relevant features for each trace
                martian_data.append({
                    'trace': filtered_trace,
                    'sta_lta': sta_lta_features,
                    'envelope': envelope,
                    'velocity': velocity,
                    'acceleration': acceleration,
                    'spectrogram': spectrogram_data,
                    'fft': fft_values,
                    'psd': psd
                })
                
                martian_time_abs.append(time_abs)  # Append absolute times for each trace
    
    return martian_data, martian_time_abs

In [49]:
from datetime import datetime

def convert_time_to_seconds(time_list, reference_time=None):
    """
    Convert a list of absolute times (e.g., '1970-01-01T12:34:56.789') to seconds since a reference time.
    If no reference time is provided, the earliest time in the list is used as the reference.
    
    Parameters:
    - time_list: List of absolute time strings.
    - reference_time: Optional. A reference time (datetime object) to calculate time differences.
    
    Returns:
    - List of time differences in seconds since the reference time.
    """
    # Flatten the time_list (in case it's a list of lists)
    flat_time_list = [item for sublist in time_list for item in sublist] if isinstance(time_list[0], list) else time_list

    # Convert the list to datetime objects
    time_list_dt = [datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.%f') for t in flat_time_list]
    
    if reference_time is None:
        # Set reference time to the earliest time in the list
        reference_time = min(time_list_dt)

    # Convert each time to seconds since the reference time
    time_in_seconds = [(t - reference_time).total_seconds() for t in time_list_dt]
    
    return time_in_seconds


In [50]:
def pad_or_truncate_sequence(sequence, target_length):
    """
    Pad the sequence with zeros or truncate it to match the target length.
    """
    if len(sequence) > target_length:
        return sequence[:target_length]  # Truncate if longer than target length
    elif len(sequence) < target_length:
        # Pad with zeros if shorter
        padding = np.zeros(target_length - len(sequence))
        return np.concatenate((sequence, padding))
    else:
        return sequence  # Return as is if already the correct length

def prepare_data_for_multitask(X, y_event, y_time, target_length=None):
    """
    Prepare data for PyTorch by padding/truncating all sequences to the same length.
    
    - X: List of input sequences (seismic traces).
    - y_event: List of event type labels.
    - y_time: List of time labels (in seconds).
    - target_length: The target length for all sequences. If None, use the length of the longest sequence.
    """
    # Find the target length for padding (either given or based on the longest sequence)
    if target_length is None:
        target_length = max([len(x) for x in X])

    # Pad or truncate all sequences to match the target length
    X_padded = [pad_or_truncate_sequence(x, target_length) for x in X]
    
    # Convert the lists to PyTorch tensors
    X_tensor = torch.tensor(X_padded, dtype=torch.float32).unsqueeze(1)  # Add channel dimension for CNN input
    y_event_tensor = torch.tensor(y_event, dtype=torch.long)  # Event type labels
    y_time_tensor = torch.tensor(y_time, dtype=torch.float32)  # Arrival time labels (now in numerical format)
    
    # Combine into a DataLoader for PyTorch
    dataset = torch.utils.data.TensorDataset(X_tensor, y_event_tensor, y_time_tensor)
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    
    return loader

In [51]:
def train_model_multitask(model, train_loader, val_loader, criterion_event, criterion_time, optimizer, num_epochs=100, device="cpu"):
    """
    Train a multitask model on the given data loaders.

    Parameters:
    - model: The neural network model to train.
    - train_loader: DataLoader for training data.
    - val_loader: DataLoader for validation data.
    - criterion_event: Loss function for event classification (cross-entropy loss).
    - criterion_time: Loss function for time prediction (MSE or MAE).
    - optimizer: Optimizer for training the model (e.g., Adam).
    - num_epochs: Number of epochs to train.
    - device: Device to use for training ('cpu' or 'cuda').
    
    Returns:
    - trained_model: The trained model after the specified number of epochs.
    """
    
    # Set the model to training mode
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, event_labels, time_labels in train_loader:
            inputs, event_labels, time_labels = inputs.to(device), event_labels.to(device), time_labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            event_output, time_output = model(inputs)

            # Compute the loss for both tasks
            loss_event = criterion_event(event_output, event_labels)
            loss_time = criterion_time(time_output.squeeze(), time_labels)  # time_output might need to be squeezed if it has an extra dimension
            loss = loss_event + loss_time

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print loss for this epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

        # Optionally, add validation loop here to monitor performance on val_loader
    
    return model


In [52]:
class CNNLSTMHybridModel(nn.Module):
    def __init__(self, input_size=1, num_classes=3, hidden_size=128, num_lstm_layers=2, dropout_prob=0.3):
        super(CNNLSTMHybridModel, self).__init__()

        # CNN layers to capture spatial patterns from seismic data
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )

        # LSTM layers to capture temporal dependencies
        self.lstm = nn.LSTM(
            input_size=64,  # Input to LSTM is the output from CNN
            hidden_size=hidden_size,
            num_layers=num_lstm_layers,
            batch_first=True,
            dropout=dropout_prob,
            bidirectional=True
        )

        # Fully connected layers for event classification and arrival time regression (multi-task learning)
        self.fc_event = nn.Sequential(
            nn.Linear(hidden_size * 2, 128),  # *2 for bidirectional
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(128, num_classes)  # Output: number of classes (3: impact, shallow, deep)
        )

        self.fc_time = nn.Sequential(
            nn.Linear(hidden_size * 2, 128),  # *2 for bidirectional
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(128, 1)  # Output: arrival time prediction (regression)
        )

    def forward(self, x):
        # Input: (batch_size, channels, seq_length) for CNN
        cnn_out = self.cnn(x)  # Shape: (batch_size, 64, reduced_seq_length)

        # Prepare input for LSTM: (batch_size, seq_length, features)
        cnn_out = cnn_out.permute(0, 2, 1)  # Shape: (batch_size, reduced_seq_length, 64)

        # LSTM forward pass
        lstm_out, _ = self.lstm(cnn_out)  # Shape: (batch_size, reduced_seq_length, hidden_size * 2)

        # Take the output from the last time step
        lstm_out = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_size * 2)

        # Event classification head
        event_output = self.fc_event(lstm_out)  # Shape: (batch_size, num_classes)

        # Arrival time regression head
        time_output = self.fc_time(lstm_out)  # Shape: (batch_size, 1)

        return event_output, time_output


In [ ]:
def main():
    # Define device (GPU or CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Define paths to lunar data
    lunar_catalog_path = '../../data/lunar_data/training/catalogs/apollo12_catalog_GradeA_final.csv'
    lunar_data_directory = '../../data/lunar_data/training/data/S12_GradeA/'

    # Load lunar data
    print("Loading and preprocessing Lunar MSEED data...")
    lunar_data, lunar_labels, lunar_time_abs = load_lunar_data(lunar_catalog_path, lunar_data_directory)
    print(f"Lunar Data Loaded: {len(lunar_data)} traces.")
    
    # Convert the absolute times (lunar_time_abs) into seconds since the start of the event
    y_time_train_seconds = convert_time_to_seconds(lunar_time_abs)

    # Ensure that the sizes match for lunar_data, lunar_labels, and y_time_train_seconds
    print(f"Data size check: {len(lunar_data)} data, {len(lunar_labels)} labels, {len(y_time_train_seconds)} times.")

    # Convert string labels in lunar_labels to integer labels using LabelEncoder
    label_encoder = LabelEncoder()
    lunar_labels_encoded = label_encoder.fit_transform(lunar_labels)

    # Split data into training and validation sets
    X_train, X_val, y_event_train, y_event_val, y_time_train, y_time_val = train_test_split(
        lunar_data, lunar_labels_encoded, y_time_train_seconds, test_size=0.2, random_state=42)

    # Prepare data for PyTorch (multitask model: event type prediction + arrival time prediction)
    train_loader = prepare_data_for_multitask(X_train, y_event_train, y_time_train)
    val_loader = prepare_data_for_multitask(X_val, y_event_val, y_time_val)

    # Initialize the CNN-LSTM Hybrid model for multitask learning
    model = CNNLSTMHybridModel().to(device)
    
    # Define optimizer and loss functions (cross-entropy for event classification, MAE for time prediction)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion_event = nn.CrossEntropyLoss()
    criterion_time = nn.MSELoss()

    # Train the model using the training and validation data
    print("Training model on Lunar dataset...")
    trained_model = train_model_multitask(model, train_loader, val_loader, criterion_event, criterion_time, optimizer, num_epochs=100, device=device)
    
    # Save the trained model
    torch.save(trained_model.state_dict(), 'pretrained_lunar_model.pth')
    print("Model training complete and saved.")

if __name__ == "__main__":
    main()